In [1]:
import pandas as pd
import numpy as np
import platform
import matplotlib.pyplot as plt
%matplotlib inline

In [2]:
from matplotlib import font_manager, rc # 한글 폰트 사용

plt.rcParams["font.family"] = "malgun Gothic"
plt.rcParams['axes.unicode_minus'] = False

In [3]:
data_map_korea = pd.read_csv('data/data_map_korea.csv', index_col=0, encoding='UTF-8')
data_map_korea.head()

,인구수,shortName,x,y,면적,광역시도,행정구역
0,202520,강릉,11,4,1040.07,강원도,강릉시
1,25589,고성(강원),9,0,664.19,강원도,고성군
2,86747,동해,11,5,180.01,강원도,동해시
3,63986,삼척,11,8,1185.80,강원도,삼척시
4,76733,속초,9,1,105.25,강원도,속초시


In [4]:
# 위 코드들을 함수로 하나로 정리


def mapping(targetData, blockedMap, d1, d2, cmapname):
    blockedMap = data_map_korea
    gamma = 0.75
    whitelabelmin = (max(blockedMap[targetData]) - min(blockedMap[targetData])) * 0.25 + min(blockedMap[targetData])
    datalabel = targetData
    vmin = min(blockedMap[targetData])
    vmax = max(blockedMap[targetData])
    BORDER_LINES = [
        [(3, 2), (5, 2), (5, 3), (9, 3), (9, 1)], # 인천
        [(2, 5), (3, 5), (3, 4), (8, 4), (8, 7), (7, 7), (7, 9), (4, 9), (4, 7), (1, 7)], # 서울
        [(1, 6), (1, 9), (3, 9), (3, 10), (8, 10), (8, 9),
         (9, 9), (9, 8), (10, 8), (10, 5), (9, 5), (9, 3)], # 경기도
        [(9, 12), (9, 10), (8, 10)], # 강원도
        [(10, 5), (11, 5), (11, 4), (12, 4), (12, 5), (13, 5),
         (13, 4), (14, 4), (14, 2)], # 충청남도
        [(11, 5), (12, 5), (12, 6), (15, 6), (15, 7), (13, 7),
         (13, 8), (11, 8), (11, 9), (10, 9), (10, 8)], # 충청북도
        [(14, 4), (15, 4), (15, 6)], # 대전시
        [(14, 7), (14, 9), (13, 9), (13, 11), (13, 13)], # 경상북도
        [(14, 8), (16, 8), (16, 10), (15, 10),
         (15, 11), (14, 11), (14, 12), (13, 12)], # 대구시
        [(15, 11), (16, 11), (16, 13)], # 울산시
        [(17, 1), (17, 3), (18, 3), (18, 6), (15, 6)], # 전라북도
        [(19, 2), (19, 4), (21, 4), (21, 3), (22, 3), (22, 2), (19, 2)], # 광주시
        [(18, 5), (20, 5), (20, 6)], # 전라남도
        [(16, 9), (18, 9), (18, 8), (19, 8), (19, 9), (20, 9), (20, 10)], # 부산시
    ]

    mapdata = blockedMap.pivot(index='y', columns='x', values=targetData)
    masked_mapdata = np.ma.masked_where(np.isnan(mapdata), mapdata)
    
    plt.figure(figsize=(8, 13))
    plt.pcolor(masked_mapdata, vmin=vmin, vmax=vmax, cmap=cmapname, edgecolor='#aaaaaa', linewidth=0.5)

    # 지역 이름 표시
    for idx, row in blockedMap.iterrows():
        annocolor = 'white' if row[targetData] > whitelabelmin else 'black'

        # 광역시는 구 이름이 겹치는 경우가 많아서 시단위 이름도 같이 표시한다. (중구, 서구)
        if row[d1].endswith('시') and not row[d1].startswith('세종'):
            dispname = '{}\n{}'.format(row[d1][:2], row[d2][:-1])
            if len(row[d2]) <= 2:
                dispname += row[d2][-1]
        else:
            dispname = row[d2][:-1]

        # 서대문구, 서귀포시 같이 이름이 3자 이상인 경우에 작은 글자로 표시한다.
        if len(dispname.splitlines()[-1]) >= 3:
            fontsize, linespacing = 9.5, 1.5
        else:
            fontsize, linespacing = 11, 1.2

        plt.annotate(dispname, (row['x']+0.5, row['y']+0.5), weight='bold',
                     fontsize=fontsize, ha='center', va='center', color=annocolor,
                     linespacing=linespacing)
        
    # 시도 경계 그린다.
    for path in BORDER_LINES:
        ys, xs = zip(*path)
        plt.plot(xs, ys, c='black', lw=4)

    plt.gca().invert_yaxis()
    #plt.gca().set_aspect(1)

    plt.axis('off')

    cb = plt.colorbar(shrink=.1, aspect=10)
    cb.set_label(datalabel)

    plt.tight_layout()
    plt.show()

In [5]:
df = pd.read_csv('data/신혼부부.csv', encoding='UTF-8', sep=',')
df.head(3)

,행정구역,신혼수_21,신혼증감_21,신혼증감률_21
0,전국,871428,-66652,-7.1
1,서울특별시,172850,-15940,-8.4
2,서울특별시 종로구,1851,-238,-11.4


In [ ]:
############## 신혼부부 행정구역을 다시 나누기(data_map_korea와 맞춤) ##########

In [15]:
korea = df['행정구역'].str.split()
korea

0                 [전국]
1              [서울특별시]
2         [서울특별시, 종로구]
3          [서울특별시, 중구]
4         [서울특별시, 용산구]
            ...       
274        [경상남도, 거창군]
275        [경상남도, 합천군]
276          [제주특별자치도]
277     [제주특별자치도, 제주시]
278    [제주특별자치도, 서귀포시]
Name: 행정구역, Length: 279, dtype: object

In [17]:
korea[4][-1]

'용산구'

In [18]:
korea[4][-1][:-1]

'용산'

In [20]:
####### 광역시도//행정구역 나누기

# 공백 기준 리스트로 만들기 -> [-1]이 "소계"이면 삭제
dist1_list= []
dist2_list= []
dist3_list= []
for i in range(len(df.index)):
    if len(korea[i]) <= 2 :  # 리스트 요소가 2개 이하이면
        dist1_list.append(korea[i][0])
        dist2_list.append(korea[i][-1])
        dist3_list.append(korea[i][-1][:-1])
    elif len(korea[i]) <= 1:
        dist3_list.append(korea[i][0])
    else:
        print(korea[i])     # 시도 1개 또는 시도//시군구 2개 요소까지만 있음


In [41]:
df['광역시도'] = dist1_list
df['행정구역'] = dist2_list
df['']

In [43]:
df.행정구역

0           전국
1        서울특별시
2          종로구
3           중구
4          용산구
        ...   
274        거창군
275        합천군
276    제주특별자치도
277        제주시
278       서귀포시
Name: 행정구역, Length: 279, dtype: object

In [44]:
df.광역시도

0           전국
1        서울특별시
2        서울특별시
3        서울특별시
4        서울특별시
        ...   
274       경상남도
275       경상남도
276    제주특별자치도
277    제주특별자치도
278    제주특별자치도
Name: 광역시도, Length: 279, dtype: object

In [21]:
df_k = data_map_korea['shortName'].tolist()

In [23]:
data_map_korea['최종행정구역'] = data_map_korea['shortName'].map(dict(zip(df_k, df_k)))